# Notebook Data Mining


## But  
Il s'agit, à partir des clusters déterminés par Kmean, d'identifier les bières similaires entre elles. Le clustering en lui-même n'apporte en effet pas directement ces informations, puisque une bière se retrouve souvent dans plusieurs clusters qui représentent des types de consommateurs. 

On pourra ensuite recommander des bières similaires aux goûts d'un consommateur.

## Démarche
Nous allons travailler sur les dataframes du clustering afin de lancer un algorithme de data mining, l'algorithme Apriori. Cet algorithme prend en entrée une liste de listes, où chacune des listes internes comprend des bières qui sont communes, c'est-à-dire des bières qu'on pourrait recommander entre elles à un consommateur sur la base de leurs similitudes déterminées à partir des avis subjectifs des consommateurs (base reviews). 

In [1]:
# Importation des modules nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df_subj_abv = pd.read_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\data_all_clusters.csv')
df_subj_abv

df_subj_abv = df_subj_abv.drop(['Unnamed: 0', 'index', 'look', 'smell', 'taste', 'feel'], axis = 'columns')
df_subj_abv = df_subj_abv.sample(10**3)

On va créer un data frame par cluster, afin de pouvoir tester les lignes de codes plus rapidement et de manière séparée.

In [3]:
cluster_all = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_all']]
cluster_subj = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_subj']]
cluster_nolook = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_nolook']]
cluster_feel_taste = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_feel/taste']]
cluster_feel_taste = cluster_feel_taste.rename(columns = {'cluster_feel/taste' : 'cluster_feel_taste'}) # Pour éliminer le problème avec le caractère '/'
cluster_taste_smell = df_subj_abv[['id', 'beer_name', 'beer_style', 'abv', 'brewery_id', 'brewery_name', 'cluster_taste/smell']]
cluster_taste_smell = cluster_taste_smell.rename(columns = {'cluster_taste/smell' : 'cluster_taste_smell'})

On crée une liste de ces dataframes, de leurs noms et un dictionnaire répertoriant le nombre de clusters de chacun des dataframes. Ils seront utiles pour l'automatisation de la mise en forme des données pour Apriori.

In [4]:
clusters_list = [cluster_all, cluster_subj, cluster_nolook, cluster_feel_taste, cluster_taste_smell]

clusters_name = ['cluster_all', 'cluster_subj', 'cluster_nolook', 'cluster_feel_taste', 'cluster_taste_smell']

clusters_nb = {}
for name, clus in zip(clusters_name, clusters_list):
    clusters_nb[name] = clus[name].nunique()
clusters_nb

{'cluster_all': 54,
 'cluster_subj': 53,
 'cluster_nolook': 26,
 'cluster_feel_taste': 15,
 'cluster_taste_smell': 14}

In [5]:
# Création de la liste de listes qui sera implémentée dans l'algorithme Apriori 
list_apriori = []

for clus, clus_name in zip(clusters_list, clusters_name):
    all_cluster = []
    c = clusters_nb[clus_name]
    
    for i in range(0, c):
        # On va travailler sur des arrays afin d'améliorer la rapidité des calculs
        dico_ex = {} # dictionnaire qui contiendra le nombre d'occurence de chaque id_beer dans chaque cluster
        listex = clus[clus[clus_name] == i].groupby('id')['id'].count() # les id_beer
        listex = np.array(listex)
        liste_id_beer = clus[clus[clus_name] == i].id.sort_values().unique() # l'occurence de chaque id_beer
        
        n = len(liste_id_beer)
        for j in range(0, n):
            dico_ex[liste_id_beer[j]] = listex[j] # création du dictionnaire 

        try:
            s = max(dico_ex[i] for i in dico_ex) # on prévient le cas où un id_beer clé n'aurait qu'une seule valeur
        except ValueError:
            break
        
        for i in range(s): # L'association sera d'autant plus forte qu'il y a d'avis correspondant à la bière dans le cluster
            nb_nan, nb_id = 0, 0
            ticket = [] # liste des id_beer associées par leur présence dans le même cluster
            for id_beer in dico_ex:
                if dico_ex[id_beer] >= 1:
                    ticket.append(str(id_beer))
                    dico_ex[id_beer] -= 1
                    nb_id += 1
                else:
                    nb_nan += 1
                
            if nb_nan <= nb_id:
                all_cluster.append(ticket) # On rempli seulement s'il y a plus de nan que de bières pour que ce soit représentatif
            else:
                break
                
    list_apriori += all_cluster

## Application de l'algorithme Apriori et mise en forme des associations des bières

Apriori est un algorithme qui permet de mettre en évidence des similarités et associations entre produits. Son application basique consiste à déterminer des produits qui sont souvent achetés ensemble dans un supermarché à l'aide de tickets de caisses. 

Nous avons repris cette idée en construisant nous-même les "tickets de caisses", qui sont en fait les sous-listes de la liste "list_apriori". Chacune de ces sous-listes représente en effet une association de bières, puisqu'elle a été construit selon la présence de chaque bière dans chacun des clusters réalisé avec K-mean.

Nous nous attendons donc à récupérer en sortie des associations de bières qui "vont ensemble", et c'est bien que l'on veut obtenir afin de pouvoir faire de la recommandation.

L'algorithme Apriori se paramètre avec 4 grandeurs :
- min_support : le support représente la proportion de chaque bière dans l'ensemble de la liste qui nourrit l'algorithme. Vu la taille de notre base de données et la quantité de bières assez importante, nous prendront cette valeur très petite (1 %) car elle n'est pas très significative.
- min_confidence : la confiance (confidence en anglais) représente la probabilité qu'une bière donnée soit dans la même sous-liste (ie le même ticket de caisse si on reprend notre exemple) qu'une autre. min_confidence sera doncla valeur de la confiance minimale qu'on va exiger pour dire que deux bières "peuvent aller ensemble". On prendra arbitrairement une confiance d'au moins 50%.
- min_lift : le lift correspond à un facteur qui représente combien de fois une bière est susceptible d'être présente dans la sous-liste en plus si elle est en présence d'une autre bière donnée. Cette grandeur est assez peu représentative ici étant donné la taille de la base de donnée et on va donc prendre la valeur habituelle 3.
- min_length : c'est le nombre minimal de bière que l'on veut pouvoir associer ensemble. On prend ici le chiffre 2. Cela signifie que l'on aura au moins 2 bière dans chaque rêgle d'association renvoyée par l'algorithme (Il est probable que l'on se limite à exploiter les associations de bières deux à deux de toutes façons).

In [6]:
#!pip install apyori
from apyori import apriori

On va appliquer l'algorithme à notre liste de listes "list_apriori". le plus complexe restera la mise en page des résultats, étant donné que l'algorithme renvoie un objet assez singulier : un générateur. 

Nous allons construire un programme qui regroupe les bières similaires dans un array pour des raisons de rapidité puis dans un dataframe pour pouvoir exporter les résultats.

In [86]:
association_rules = apriori(list_apriori, min_support=0.01, min_confidence=0.5, min_lift=3, min_length=2)

S = 0
for k in list_apriori:
    for i in k:
        S += 1
        
# Il y a S bières dans la liste qui va entraîner Apriori        
# Dans le pire des cas, soit il n'y a aucune association entre les bières sélectionnées
# Soit elles sont toutes associées
association_results = np.empty((S, S), dtype = int)
twin = [0, 1]
ind_max, col_max = 0, 0# Pour être sûr insérer la nouvelle bière dans une case innoccupée

for k in range(S): # Dans le pire des cas, on aura associé toutes les S bières au bout de S itérations (apriori ne fait pas de doublons)
    try:
        frozen = next(association_rules)[0]
    except StopIteration:
        pass
    associated = np.zeros(2)
    for (l,m) in zip(frozen, twin):
        associated[m] = l
    id1, id2 = associated[0], associated[1]
    
    
    
    flag = 0
    for i in range(col_max):
        if id1 in association_results[i]:
            association_results[i, ind_max] = id2
            ind_max += 1
            break
        
        elif id2 in association_results[i]:
            association_results[i, ind_max] = id1
            ind_max += 1
            break
            
        else:
            flag += 1 
            

    if flag == col_max:
        association_results[col_max, 0], association_results[col_max, 1] =  id1, id2
        col_max += 1



association_results = np.transpose(association_results) 
df_association_results = pd.DataFrame(association_results)

df_association_results.to_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\association_rules.csv')
df_association_results

,0,1,2,3,4,5,6,7,8,9,...,4244,4245,4246,4247,4248,4249,4250,4251,4252,4253
0,101052,1053,1010,113523,100421,102,1013,105038,10274,117218,...,0,0,0,0,0,0,0,0,0,0
1,101808,100,100040,1003,10672,101096,102519,101981,118967,103016,...,0,0,0,0,0,0,0,0,0,0
2,0,105503,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1206,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,122594,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


La fonction apriori renvoie un objet de type générateur. Ce type d'objet sert à créer des itérateur et on accède à ces éléments avec l'instruction next. On trouva à l'intérieur de ces itération deux informations intéressantes : 
- un frozenset, type non-mutable, qui contient les deux bières associées, repérées par leur id
- un float qui contient un coefficient représentatif de l'associativité des 2 bières du fronzenset.

On a créé un DataFrame avec pour chaque id_beer sélectionné, toutes les bières qui lui sont associées en colonne. On a déjà imposé une confidence minimum de 50% et on ne va donc pas se préoccuper de l'association particulière entre chaque bière pour l'instant.

## Recommandation de bières

On s'approche du but, qui est de recommander à un consommateur des bières proches de ses goûts. 

On va proposer un échantillon aléatoire des bières du dataframe des bières "similaires", df_simil, au consommateur. Il les note et on lui propose des bières similaires à découvrir.

#### Piste d'approfondissement :
- Proposer à la notation les bières les plus connues à l'aide du nombre d'avis
- Proposer au consommateur de noter les bières de manière plus détaillée selon (look, smell, taste, feel) afin de pouvoir déterminer son type par l'intermédiaire d'une classification dans les clusters que nous avions trouvé.


In [87]:
# Importation des données à utiliser pour la recommandation

df_simil = pd.read_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\association_rules.csv')

df_subj_abv = pd.read_csv(r'C:\Users\sim13\OneDrive\Documents\Projet_python_2A\data_avis_subj.csv')
df_subj_abv = df_subj_abv
df_subj_abv

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,4244,4245,4246,4247,4248,4249,4250,4251,4252,4253
0,0,101052,1053,1010,113523,100421,102,1013,105038,10274,...,0,0,0,0,0,0,0,0,0,0
1,1,101808,100,100040,1003,10672,101096,102519,101981,118967,...,0,0,0,0,0,0,0,0,0,0
2,2,0,105503,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1206,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,122594,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,4249,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4250,4250,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4251,4251,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4252,4252,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Recommandation aléatoire

La fonction ci-dessous propose une bière à l'utilisateur de manière aléatoire, lui demande s'il souhaite en découvrir une similaire et dès que la réponse est positive, exécute la requête du consommateur et lui proposant, de façon également aléatoire, une bière similaire à découvrir !

In [118]:
from random import * 


proposal = False
while not proposal:
    # On tire 5 bières au hasard parmi celles qui ont été traitées par l'algorithme Apriori
    ech_note = sample(list_apriori[randint(0, len(list_apriori))], 1)
    
    name = df_subj_abv[df_subj_abv['id'] == int(beer)]['beer_name'].unique()[0]
    # On trouve leur nom dans df_subj_abv
    df_subj_abv[df_subj_abv['id'] == int(beer)]['beer_name'].unique()[0]
    notes = {}
    for beer in ech_note:
        knowledge = input("Souhaiteriez-vous découvrir une bière similaire à celle-ci ? (yes/no) : " + str(name))
        if knowledge == 'yes':
            id_simil = df_subj_abv[df_subj_abv['beer_name'] == str(name)]['id'].unique()[0]
            arr_simil = np.array(df_simil)
            n, m = np.shape(arr_simil)

            for i in range(n):
                if id_simil in arr_simil[i]:
                    decouverte_id = sample(list(arr_simil[i]), 1)
                    while decouverte_id[0] == 0 or decouverte_id == id_simil:
                        decouverte_id = sample(list(arr_simil[i]), 1)
                    decouverte_name = df_subj_abv[df_subj_abv['id'] == int(decouverte_id[0])]['beer_name'].unique()[0]
            print("Vous devriez essayer : " + str(decouverte_name))
            
            proposal = True
            break
        else:
            pass
    


Souhaiteriez-vous découvrir une bière similaire à celle-ci ? (yes/no) : 15 Feetnp
Souhaiteriez-vous découvrir une bière similaire à celle-ci ? (yes/no) : Oatmeal Cookie Aleno
Souhaiteriez-vous découvrir une bière similaire à celle-ci ? (yes/no) : Agave Wheatyes
Vous devriez essayer : Imperial Or Xata
